In [1]:
import ast
import os

os.chdir('/data/hrz/LibraryMigrationPy')

In [2]:
with open('setup-test.py', 'r') as f:
    tree = ast.parse(f.read())
ast.dump(tree)

"Module(body=[Expr(value=Constant(value='\\n@author: HJK \\n@file: setup.py \\n@time: 2019-01-26\\n\\n打包配置文件\\n\\n', kind=None)), Import(names=[alias(name='os', asname=None)]), Import(names=[alias(name='sys', asname=None)]), Import(names=[alias(name='setuptools', asname=None)]), If(test=Compare(left=Subscript(value=Attribute(value=Name(id='sys', ctx=Load()), attr='argv', ctx=Load()), slice=Index(value=UnaryOp(op=USub(), operand=Constant(value=1, kind=None))), ctx=Load()), ops=[Eq()], comparators=[Constant(value='publish', kind=None)]), body=[Expr(value=Call(func=Attribute(value=Name(id='os', ctx=Load()), attr='system', ctx=Load()), args=[Constant(value='rm -rf dist', kind=None)], keywords=[])), Expr(value=Call(func=Attribute(value=Name(id='os', ctx=Load()), attr='system', ctx=Load()), args=[Constant(value='python setup.py sdist bdist_wheel', kind=None)], keywords=[])), Expr(value=Call(func=Attribute(value=Name(id='os', ctx=Load()), attr='system', ctx=Load()), args=[Constant(value='twin

In [3]:
# read credentials from config
import configparser

config = configparser.ConfigParser()
if (os.path.exists('config.ini')):
    config.read('config.ini')
else:
    raise Exception('config.ini not found')
    
try:
    github_proxy=config['Proxy']['proxy']
except KeyError:
    github_proxy= None

In [4]:
# parse setup.py with ast (deprecated)
# developers can do literally anything in setup.py
# makes it hard to parse setup.py

# import textwrap

# def parse_setup(setup_filename):
#     """Parse setup.py and return args and keywords args to its setup
#     function call

#     """
#     mock_setup = textwrap.dedent('''\
#     def setup(*args, **kwargs):
#         __setup_calls__.append((args, kwargs))
#     ''')
#     parsed_mock_setup = ast.parse(mock_setup, filename=setup_filename)
#     with open(setup_filename, 'rt') as setup_file:
#         parsed = ast.parse(setup_file.read())
#         for index, node in enumerate(parsed.body[:]):
#             if (
#                 not isinstance(node, ast.Expr) or
#                 not isinstance(node.value, ast.Call) or
#                 node.value.func.value.id != 'setup'
#             ):
#                 continue
#             print(node.value)
#             parsed.body[index:index] = parsed_mock_setup.body
#             break

#     fixed = ast.fix_missing_locations(parsed)
#     codeobj = compile(fixed, setup_filename, 'exec')
#     local_vars = {}
#     global_vars = {'__setup_calls__': []}
#     exec(codeobj, global_vars, local_vars)
    
#     # no setup calls
#     if not global_vars['__setup_calls__']:
#         return []
#     else:
#         return global_vars['__setup_calls__'][0]

# parse_setup('setup-test.py')

In [5]:
os.chdir('/data/hrz/LibraryMigrationPy')
REPOS_DIR = 'repos/'
if not os.path.exists(REPOS_DIR):
    os.mkdir(REPOS_DIR)
REPOS_DIR=os.path.abspath(REPOS_DIR)
REPOS_DIR

'/data/hrz/LibraryMigrationPy/repos'

In [6]:
import os
import sys
from distutils.core import run_setup
from utils.download_from_github import download_file_from_github

def setup_runner(owner: str, repo: str, branch='master', setup_file='setup.py', REPOS_DIR='repos/', max_tries=10):
    
    oldcwd = os.getcwd()
    path = os.path.abspath(f'{REPOS_DIR}/{owner}/{repo}/{branch}/')
    os.chdir(path)
    
    result = None
    for _ in range(max_tries):
        try:
            # why run_setup depends on cwd
            result = run_setup(setup_file, stop_after="init")
            break
        except Exception as e:
            # file not found, download on demand
#             print(f'Missing {owner}/{repo}/{e.filename}, downloading')
            if isinstance(e, FileNotFoundError):
                download_file_from_github(owner=owner, repo=repo, branch=branch, file=e.filename, 
                                          proxy=github_proxy, REPOS_DIR=REPOS_DIR)
            else:
                # fail silently
                break
                
    os.chdir(oldcwd)
    assert(os.getcwd() == oldcwd)
    return result.install_requires if result else None

setup_runner('hall-lab','svtyper', REPOS_DIR=REPOS_DIR)

/home/hrz/miniconda3/lib/python3.8/site-packages/setuptools/dist.py:463: UserWarning: Normalizing 'v0.7.1' to '0.7.1'
  warnings.warn(tmpl.format(**locals()))


['pysam>=0.15.0', 'numpy', 'scipy', 'cytoolz>=0.8.2']

In [7]:
# ref: https://codereview.stackexchange.com/questions/25417/
# to install silently
class NoStdStreams(object):
    def __init__(self,stdout = None, stderr = None):
        self.devnull = open(os.devnull,'w')
        self._stdout = stdout or self.devnull or sys.stdout
        self._stderr = stderr or self.devnull or sys.stderr

    def __enter__(self):
        self.old_stdout, self.old_stderr = sys.stdout, sys.stderr
        self.old_stdout.flush(); self.old_stderr.flush()
        sys.stdout, sys.stderr = self._stdout, self._stderr

    def __exit__(self, exc_type, exc_value, traceback):
        self._stdout.flush(); self._stderr.flush()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr
        self.devnull.close()

In [8]:
def setup_parser_wrapper(project: str) -> bool:
    repo=project.split('/')[1]
    owner=project.split('/')[0]
    branch='master'
    
    result = None
    if os.path.exists(f"{REPOS_DIR}/{owner}/{repo}/{branch}/setup.py"):
        with NoStdStreams():
            result = setup_runner(owner=owner,repo=repo, REPOS_DIR=REPOS_DIR)
        
    if result:
        with open(f"{REPOS_DIR}/{owner}/{repo}/{branch}/requirements-setup.txt", 'w+') as f:
            f.writelines(s + '\n' for s in result)  # add \n
        return True
    return False

setup_parser_wrapper('hall-lab/svtyper')

True

In [9]:
import pandas as pd
df_projects_w_requirements = pd.read_csv('data/projects.csv', low_memory=False)
projects_w_requirements = df_projects_w_requirements['Name with Owner'].to_list()
f"{len(projects_w_requirements)} projects with head {projects_w_requirements[0]}"

'14279 projects with head vinta/awesome-python'

In [ ]:
from utils.parallel import parallel
from tqdm.notebook import tqdm

parallel(setup_parser_wrapper, 8, projects_w_requirements, progress_bar=tqdm, return_df=False)

计算进度:   0%|          | 0/14279 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

In [ ]:
for row in df_projects_w_requirements.iterrows():
    project = row['Name with Owner']
    repo=project.split('/')[1]
    owner=project.split('/')[0]
    branch='master'
    if os.path.exists(f"{REPOS_DIR}/{owner}/{repo}/{branch}/requirements-setup.txt"):
        row['setup_parsed'] = True
    else:
        row['setup_parsed'] = False
df_projects_w_requirements

In [ ]:
df_projects_w_requirements.to_csv('data/projects.csv', index=False)